# Simpy Exercises 2:

## Modelling more complex pathways

In [1]:
import simpy
simpy.__version__

'3.0.12'

In [2]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt

# Modelling a minor injury unit

The exercises in this notebook will focus on a model of a minor injury unit (MIU).  In the UK, the NHS provides MIU's provide assessment and treatment for non-life threatening injuries.  For example, cuts and broken bones. 

The process flow below displays a high level process map of a MIU.  
* Patients arrive following a Poisson process, wait for nurse triage in one of **three triage bays**. 
* After triage is complete the patient waits for an assessment and treatment cubicle (there are **six**) where they are attended to by a nurse (here we assume the nurse is not a rate limiting step).  
* Around **45%** of patients require diagnostics (e.g. an x-ray or other imaging) in addition to an assessment.  
* Around **75%** of diagnostics tests are booked by the nurse at the start of consultation with the remaining beginning their wait after the initial assessment.
* Patients who undergo diagnostics are seen by a nurse for a 2nd time following the completion of their diagnostics.

The duration of activities are:

| Activity 	| Activity name        	| Distribution 	| Mean (mins) 	| Standard Dev (minutes) 	|
|----------	|----------------------	|--------------	|-------------	|------------------------	|
| 1        	| IAT                  	| Exponential  	| 5.0          	|                        	|
| 2        	| 1st Assessment            	| Lognormal        	| 10          	| 3                      	|
| 3       	| Diagnostic waiting time  	| Exponential        	| 30          	|                       	|
| 4       	| 2nd Asessment  	| Lognormal        	| 10          	| 5                      	|


| Activity 	| Activity name        	| Distribution 	| Low 	| Mode 	| High |
|----------	|----------------------	|--------------	|-------------	|------------------------	| |
| 5        	| Triage                  	| Triangular  	| 2.0          	|   5.0                     	| 10.0|
| 6        	| Diagnostics           	| Triangular        	| 10.0          	| 15.0                      	|20.0 |




![image](images/minor_injury_process_flow.png)

# Distribution classes

To help you build your model, the notebook includes some pre-written distribution classes that you may wish to use to setup sampling.  You are free to use these, but you can choose not too if you have an approach you prefer. 

In [83]:
class Exponential():
    '''
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        mean: float
            The mean of the exponential distribution
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.exponential(self.mean, size=size)

class Triangular():
    '''
    Convenience class for the triangular distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, low, mode, high, random_seed=None):
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high
        self.mode = mode
        
    def sample(self, size=None):
        return self.rand.triangular(self.low, self.mode, self.high, size=size)
    
class Bernoulli():
    '''
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, p, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        p: float
            probability of drawing a 1
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.p = p
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.binomial(n=1, p=self.p, size=size)

class Lognormal(object):
    """
    Encapsulates a lognormal distirbution
    """
    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean = mean of the lognormal distribution
        stdev = standard dev of the lognormal distribution
        """
        self.rand = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma
        
    def normal_moments_from_lognormal(self, m, v):
        '''
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m = mean of lognormal distribution
        v = variance of lognormal distribution
                
        Returns:
        -------
        (float, float)
        '''
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2/phi)
        sigma = math.sqrt(math.log(phi**2/m**2))
        return mu, sigma
        
    def sample(self):
        """
        Sample from the normal distribution
        """
        return np.random.lognormal(self.mu, self.sigma)

# Utility functions

In [81]:
def trace(msg):
    '''
    Utility function for printing simulation
    set the TRACE constant to FALSE to 
    turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

# Model parameters and scenario class

* The constants below provides hard coded data representing the base case or 'as-is' state of the minor injury unit.   
* In the cell below the parameters you will find a `Scenario` class.  This makes use of the default parameters to set up the base case scenario.  Remember that its good practice to pass all of your parameters to your simulation model in a **container**.  A class is a flexible way to achieve this aim.

In [86]:
# These are the parameters for a base case model run.
#Note if you change these parameters then your model will run a new 'scenario' 

#resource counts
N_CUBICLES = 6
N_BAYS = 2

#time between arrivals in minutes (exponential)
MEAN_IAT = 5

#triage (triangular)
TRIAGE_LOW = 2
TRIAGE_MODE = 5
TRIAGE_HIGH = 10

#assessment (lognormal)
ASSESS_MEAN = 10
ASSESS_STD = 3

#2nd assessment (lognormal)
ASSESS_MEAN_2 = 10
ASSESS_STD_2 = 5

#diagnostics (bernoulli)
PROB_DIAG = 0.45 

#diagnostics waiting time (exp)
DIAG_WT_MEAN = 30

#diagnostics process time (triangular)
DIAG_PT_LOW = 10
DIAG_PT_MODE = 15
DIAG_PT_HIGH = 20

#% in parallel (bernoulli)
P_PARALLEL = 0.75

#SEEDS to reproduce results of a single run
REPRODUCIBLE_RUN = True
    
if REPRODUCIBLE_RUN:
    SEEDS = [42, 101, 1066, 1966, 2013, 999, 1444, 2016]
else:
    SEEDS = [None, None, None, None, None, None, None]

In [87]:
class Scenario(object):
    '''
    Parameter container class for minor injury unit model.
    '''
    def __init__(self):
        '''
        The init method sets up our defaults. 
        '''
        
        #triage bays
        self.triage_bays = simpy.Resource(env, capacity=N_BAYS)
        
        #assessment and treatment cubicles
        self.minor_cubicles = simpy.Resource(env, capacity=N_CUBICLES)
        
        #inter-arrival distribution
        self.arrival_dist = Exponential(MEAN_IAT, random_seed=SEEDS[0])
        
        #triage distribution
        self.triage_dist = Triangular(TRIAGE_LOW, TRIAGE_MODE, TRIAGE_HIGH, 
                                      random_seed=SEEDS[1])
        
        #assessment distribution
        self.assessment_dist = Lognormal(ASSESS_MEAN, ASSESS_STD, 
                                         random_seed=SEEDS[2])
        
        #diagnostics: prob that patient needs imaging etc.
        self.n_diagnostics_minor = Bernoulli(PROB_DIAG, random_seed=SEEDS[3])
        
        #waiting and process time dists for diagnostics
        self.diag_wait_dist = Exponential(DIAG_WT_MEAN, random_seed=SEEDS[4])
        self.diag_dist = Triangular(DIAG_PT_LOW, DIAG_PT_MODE, DIAG_PT_HIGH,
                                    random_seed=SEEDS[5])
        
        #prob diagnostics done in parallel to nurse assessment
        self.p_diag_parallel = Bernoulli(P_PARALLEL, random_seed=SEEDS[6])
        
        #2nd assessment distribution
        self.assessment_dist = Lognormal(ASSESS_MEAN_2, ASSESS_STD_2, 
                                         random_seed=SEEDS[7])


# Exercise: Setting up the model process logic





In [75]:
class MinorPatient(object):
    '''
    Patient in the minor ED process
    '''
    def __init__(self, identifier, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        #patient id and environment
        self.identifier = identifier
        self.env = env

        #triage parameters
        self.triage_bays = args.triage_bays
        self.triage_dist = args.triage_dist
    
        #minor assessment parameters
        self.minor_cubicles = args.minor_cubicles
        self.assessment_dist = args.assessment_dist
        
        #diagnostics: prob that patient needs imaging etc.
        self.n_diagnostics_dist = args.n_diagnostics_minor
        
        #prob diagnostics done in parallel to nurse assessment
        self.p_diag_parallel = args.p_diag_parallel
        
        #waiting and process time dists for diagnostics
        self.diag_wait_dist = args.diag_wait_dist 
        self.diag_dist = args.diag_dist
                
        # individual patient metrics
        self.time_to_nurse = 0.0
        self.time_in_system = 0.0
        self.four_hour_target = 0
        
    
    def assessment(self):
        '''
        simulates the process for minor emergencies
        
        1. request and wait for triage
        2. request and wait for a minor cubicle
        3. minor assessment
        4. exit system.
        
        '''
        #record the time that patient entered the system
        arrival_time = env.now

        #request an operator 
        with self.triage_bays.request() as req:
            yield req
            
            trace(f'minor triage {self.identifier} at {self.env.now:.3f}')
            
            #sample triage duration.
            triage_duration = self.triage_dist.sample()
            yield self.env.timeout(triage_duration)
            
            trace(f'triage ended {self.identifier} ended {self.env.now:.3f}; '
                      + f' waiting time was {self.env.now - arrival_time:.3f}')
            
        
        #get cubicle
        with self.minor_cubicles.request() as req:
            yield req
            
            #record time to first being seen by a nurse
            self.time_to_nurse = self.env.now - arrival_time
            
            trace(f'minor assessment {self.identifier} at {self.env.now:.3f}; '
                      + f' time to nurse was {self.time_to_nurse:.3f}')
            
            #sample for patient pathway
            assess_time, n_diagnostics, \
                parallel, diag_wt, diag_pt = self.sample_all()
            
            #patient undergoing diagnostics?
            if n_diagnostics > 0:

                #in parallel to minor assessment?
                if parallel:
                    
                    trace(f'parallel diagnostics patient {self.identifier}')
                    
                    #use the longest delay
                    activity_duration = max(assess_time, diag_wt + diag_pt)
                    
                else:
                    
                    trace(f'diagnostics following assessment for '
                          + f'patient {self.identifier}')
                    
                    #use the total delay
                    activity_duration = assess_time + diag_wt + diag_pt
            else:
                activity_duration = assess_time
                
            yield self.env.timeout(activity_duration)
            
            trace(f'completed {self.identifier} at {self.env.now:.3f}; '
                      + f' time in system{self.time_to_nurse:.3f}')
            
            #time in system for this patient
            self.time_in_system = self.env.now - arrival_time
            
            if self.time_in_system <= (4 * 60):
                self.four_hour_target = 1
            
                        
    def sample_all(self):
        '''
        Sample assessment time, if the patient requires diagnostics,
        if the diags are done in parallel to assessment and 
        '''

        #sample assessment time
        assess_time = self.assessment_dist.sample()

        #will the patient need diagnostics?
        n_diagnostics = self.n_diagnostics_dist.sample()

        if n_diagnostics > 0:
            #will the diagnostics be in parallel with assessment
            parallel = self.p_diag_parallel.sample()

            #diagnostic waiting time
            diag_wt = self.diag_wait_dist.sample()

            #diagnostic process time
            diag_pt = self.diag_dist.sample()

            return assess_time, n_diagnostics, parallel, diag_wt, diag_pt
        else:
            return assess_time, n_diagnostics, _, _, _
    

In [76]:
class MinorInjuryUnit(object):  
    '''
    Model of an minor injury unit
    '''
    def __init__(self, env, args):
        '''
        Contructor
        
        Params:
        -------
        env: simpy.Environment
        
        args: Scenario
            container class for simulation model inputs.
        '''
        self.env = env
        self.args = args 
        
        self.patients = []
            
    def arrivals_generator(self):
        '''
        IAT is exponentially distributed

        Parameters:
        ------
        env: simpy.Environment

        args: Scenario
            Container class for model data inputs
        '''
        for patient_count in itertools.count(start=1):
            inter_arrival_time = self.args.arrival_dist.sample()
            yield env.timeout(inter_arrival_time)

            trace(f'patient {patient_count} arrives at: {env.now:.3f}')

            new_patient = MinorPatient(patient_count, self.env, self.args)
            self.patients.append(new_patient)
            
            env.process(new_patient.assessment())

In [77]:
# Script to run the model

In [79]:
#run length in minutes
RUN_LENGTH = 1440
MEAN_IAT = 5
TRACE = False

#create simpy environment
env = simpy.Environment()
#base case scenario with default parameters
default_args = Scenario()

#create the model
model = MinorInjuryUnit(env, default_args)

#setup the process
env.process(model.arrivals_generator())

env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

# Performance metrics

#1. mean time in system
mean_time_in_system = np.array([patient.time_in_system 
                                for patient in model.patients]).mean()


#2. mean time to first treatment
mean_time_to_nurse = np.array([patient.time_to_nurse 
                               for patient in model.patients]).mean()


#3. four hour target
four_hours = np.array([patient.four_hour_target 
                       for patient in 
                       model.patients]).sum() / len(model.patients)

print('\nSingle run results\n------------------')
print(f'Mean Minutes in System: {mean_time_in_system:.2f}')
print(f'Mean Minutes to Nurse: {mean_time_to_nurse:.2f}')
print(f'Proportion discharged before 4 hrs: {four_hours:.2f}')

end of run. simulation clock time = 1440.0

Single run results
------------------
Mean Minutes in System: 84.21
Mean Minutes to Nurse: 58.97
Proportion discharged before 4 hrs: 0.94


In [ ]:
TRACE = True